<a href="https://colab.research.google.com/github/pratik-poudel/financial_news_stock/blob/master/to-do.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Bidirectional,LSTM, GlobalMaxPool1D, Dense



from tensorflow.keras.utils import to_categorical

In [54]:
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", 
                                with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test", 
                      as_supervised=True)

In [55]:
imdb_train = tfds.as_dataframe(imdb_train, ds_info)
imdb_test = tfds.as_dataframe(imdb_test, ds_info)

In [68]:
imdb_train = imdb_train.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
imdb_test = imdb_test.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

In [112]:
X_train = imdb_train['text']
y_train = imdb_train['label']
X_test = imdb_test['text']
y_test = imdb_test['label']

In [88]:
tokenizer= Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test= tokenizer.texts_to_sequences(X_test)

In [91]:
word_index = tokenizer.word_index

In [98]:
max_len = np.max([len(X_train[i]) for i in range(len(X_train))])

In [ ]:
Tokenizer()

In [102]:
X_train = pad_sequences(X_train, maxlen=150, padding='post', )
X_test = pad_sequences(X_test, maxlen = 150, padding='post')

In [106]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [115]:
n_classes = 1
def build_model_lstm(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
    mask_zero=True),
    tf.keras.layers.LSTM(rnn_units),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model



In [116]:
vocab_size = len(word_index) +1
# The embedding dimension
embedding_dim = 64
# Number of RNN units
rnn_units = 64
# batch size
BATCH_SIZE=100
model = build_model_lstm(
    vocab_size = vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          5669312   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,702,401
Trainable params: 5,702,401
Non-trainable params: 0
_________________________________________________________________


In [118]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', 'Precision', 'Recall'])

model.fit(X_train, y_train, epochs=10, batch_size=BATCH_SIZE, verbose=1)


Epoch 1/10
250/250 [==============================] - 13s 42ms/step - loss: 0.5405 - accuracy: 0.7022 - precision: 0.7034 - recall: 0.7006
Epoch 2/10
250/250 [==============================] - 11s 42ms/step - loss: 0.1774 - accuracy: 0.9366 - precision: 0.9344 - recall: 0.9400
Epoch 3/10
250/250 [==============================] - 10s 42ms/step - loss: 0.0855 - accuracy: 0.9739 - precision: 0.9753 - recall: 0.9726
Epoch 4/10
250/250 [==============================] - 10s 42ms/step - loss: 0.0499 - accuracy: 0.9856 - precision: 0.9844 - recall: 0.9867
Epoch 5/10
250/250 [==============================] - 10s 42ms/step - loss: 0.0265 - accuracy: 0.9928 - precision: 0.9938 - recall: 0.9919
Epoch 6/10
250/250 [==============================] - 10s 42ms/step - loss: 0.0303 - accuracy: 0.9909 - precision: 0.9922 - recall: 0.9897
Epoch 7/10
250/250 [==============================] - 10s 42ms/step - loss: 0.0358 - accuracy: 0.9872 - precision: 0.9885 - recall: 0.9856
Epoch 8/10
250/250 [=======

In [119]:
model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)

250/250 [==============================] - 3s 7ms/step - loss: 0.8269 - accuracy: 0.8362 - precision: 0.8356 - recall: 0.8370


[0.826852023601532, 0.8361600041389465, 0.8355693817138672, 0.8370400071144104]

In [127]:
model.predict(X_test[0])

0.47636515299479165

In [131]:
y_train = to_categorical(y_train)
y_test= to_categorical(y_test)

In [132]:
EMBEDDING_DIM = 300
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_len))
model.add(Dropout(0.25))
model.add(Bidirectional(LSTM(200, return_sequences=True), merge_mode='concat'))
model.add(Dropout(0.25))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics='accuracy')

In [135]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2493, 300)         26574900  
_________________________________________________________________
dropout (Dropout)            (None, 2493, 300)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 2493, 400)         801600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 2493, 400)         0         
_________________________________________________________________
global_max_pooling1d (Global (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                20050     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)               

In [134]:
batch_size = 64
epochs = 10
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,verbose=1)

Epoch 1/10
391/391 [==============================] - 35s 91ms/step - loss: 0.2858 - accuracy: 0.8836
Epoch 2/10
391/391 [==============================] - 35s 90ms/step - loss: 0.2051 - accuracy: 0.9219
Epoch 3/10
391/391 [==============================] - 35s 90ms/step - loss: 0.1554 - accuracy: 0.9426
Epoch 4/10
391/391 [==============================] - 35s 90ms/step - loss: 0.1183 - accuracy: 0.9592
Epoch 5/10
391/391 [==============================] - 35s 90ms/step - loss: 0.0844 - accuracy: 0.9712
Epoch 6/10
391/391 [==============================] - 35s 90ms/step - loss: 0.0571 - accuracy: 0.9808
Epoch 7/10
391/391 [==============================] - 35s 90ms/step - loss: 0.0411 - accuracy: 0.9863
Epoch 8/10
391/391 [==============================] - 36s 91ms/step - loss: 0.0280 - accuracy: 0.9912
Epoch 9/10
391/391 [==============================] - 35s 91ms/step - loss: 0.0179 - accuracy: 0.9943
Epoch 10/10
391/391 [==============================] - 35s 91ms/step - loss: 0.012

In [136]:
model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)

250/250 [==============================] - 4s 13ms/step - loss: 0.6060 - accuracy: 0.8502


[0.6059731841087341, 0.8501999974250793]

In [137]:
model.predict(X_test[0])

array([[3.94611120e-01, 6.05388880e-01],
       [7.62499988e-01, 2.37499967e-01],
       [9.99360025e-01, 6.39992417e-04],
       [2.54454720e-03, 9.97455418e-01],
       [2.42744815e-02, 9.75725532e-01],
       [5.18489957e-01, 4.81509984e-01],
       [1.93062499e-01, 8.06937516e-01],
       [8.01742554e-01, 1.98257491e-01],
       [6.38091326e-01, 3.61908704e-01],
       [4.71223384e-01, 5.28776646e-01],
       [8.80347371e-01, 1.19652607e-01],
       [3.91476555e-04, 9.99608576e-01],
       [9.98337388e-01, 1.66256400e-03],
       [3.48617882e-01, 6.51382089e-01],
       [5.72086036e-01, 4.27913964e-01],
       [9.84464467e-01, 1.55354915e-02],
       [1.09254186e-04, 9.99890685e-01],
       [1.99941069e-01, 8.00058901e-01],
       [8.08977187e-01, 1.91022769e-01],
       [5.03213584e-01, 4.96786386e-01],
       [8.48092139e-01, 1.51907936e-01],
       [9.69207048e-01, 3.07929907e-02],
       [5.03213584e-01, 4.96786386e-01],
       [8.48092139e-01, 1.51907936e-01],
       [6.921743